# Importando as libraries

In [18]:
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import unicodedata
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from nltk.stem import SnowballStemmer
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import glob

# Lendo e salvando todos os arquivos de trabalhadores em um único dataframe

In [3]:
path = r'SINE/Trabalhadores' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=";", encoding='iso-8859-1')
    li.append(df)

trabalhadores = pd.concat(li, axis=0, ignore_index=True)

C:\Users\bl_vi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Criando todo o framework para o preprocessamento

In [19]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [20]:
CONTRACTION_MAP = {
    "d'aquela": "de aquela",
    "d'aquella": "de aquela",
    "d'aquellas": "de aquelas",
    "d'aquelle": "de aquele",
    "d'aquelles": "de aqueles",
    "d'aquillo": "de aquilo",
    'daquela': "de aquela",
    'daquelas': "de aquelas",
    'daquele': "de aquele",
    'daqueles': "de aqueles",
    'daquella': "de aquela",
    'daquelle': "de aquele",
    'daquelles': "de aquela",
    'daqueloutro': "de aquele outro",
    'daquillo': "de aquilo",
    'daquilo': "de aquilo",
    "n'aquella": "em aquela",
    "n'aquellas": "em aquelas",
    "n'aquelle": "em aquele",
    "n'aquelles": "em aqueles",
    "n'aquillo": "em aquilo",
    'naquela': "em aquela",
    'naquelas': "em aquelas",
    'naquele': "em aquele",
    'naqueles': "em aqueles",
    'naquella': "em aquela",
    'naquelle': "em aquele",
    'naquilo': "em aquilo",
    "d'essa": "de essa",
    "d'essas": "de essas",
    "d'esse": "de esse",
    "d'esses": "de essa",
    "d'esta": "de esta",
    "d'estas": "de estas",
    "d'este": "de este",
    "d'estes": "de estes",
    'dessa': "de essa",
    'dessas': "de essas",
    'desses': "de esses",
    'desta': "de esta",
    'destas': "de estas",
    'deste': "de este",
    'destes': "de estes",
    "n'essa": "em essa",
    "n'essas": "em essas",
    "n'esse": "em esse",
    "n'esses": "em esses",
    "n'esta": "em esta",
    "n'estas": "em estas",
    "n'este": "em este",
    "n'estes": "em estes",
    'nessa': "em essa",
    'nessas': "em essas",
    'nesse': "em esse",
    'nesses': "em esses",
    'nesta': "em esta",
    'nestas': "em estas",
    'neste': "em este",
    'nestes': "em estes",
    "d'isso": "de isso",
    "d'isto": "de isto",
    'disso': "de isso",
    'disto': "de isto",
    "n'isso": "em isso",
    "n'isto": "em isto",
    'nisso': "em isso",
    'nisto': "em isto"
}

In [21]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    
    def expand_match(contraction):
        match = contraction.group(0)
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        expanded_contraction = expanded_contraction[0:]
        return expanded_contraction
    
    expanded_text = contractions_pattern.sub(expand_match, text)
    return expanded_text

In [22]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s\-]' if not remove_digits else r'[^a-zA-z\s\-]'
    text = re.sub(pattern, '', text)
    return text

In [23]:
def caseconversion(text, case_type = 'lower'):
    if case_type == 'lower':
        text = text.lower()
    elif case_type == 'upper':
        text = text.upper()
    elif case_type == 'title':
        text = text.title()
    return text

In [24]:
def simple_stemmer(text, stemmer='RSLP'):
    ps = RSLPStemmer()
    if stemmer=='Snowball':
        ps = SnowballStemmer("portuguese")
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [25]:
stopword_list = nltk.corpus.stopwords.words('portuguese')
def remove_stopwords(text, is_lower_case=False):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [26]:
def preprocess_corpus(corpus, contraction_expansion=True, accented_char_removal=True, 
                     text_lower_case=True, stem_or_lemma='stem', special_char_removal=True, stopword_removal=True, 
                     remove_digits=False):
    preprocessed_corpus = []
    # preprocess each document in the corpus
    for doc in tqdm(corpus):
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
            doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize or stem text
#         if stem_or_lemma == 'stem':
#             doc = simple_stemmer(doc, stemmer='RSLP')
#         elif stem_or_lemma == 'lemma':
#             doc = lemmatize_text(doc)
#         else doc = doc
        # remove special characters and\or digits
        if special_char_removal:
            # insert spaces between special characters to isolate them
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
        preprocessed_corpus.append(doc)
    return preprocessed_corpus

# Criando um dataframe apenas com trabalhadores que cadastraram cursos profissionalizantes

In [44]:
trab_com_cursos = trabalhadores[trabalhadores.CURSOS_PROFISSIONALIZANTES.notnull()]

# Explodindo o campo PRETENSOES em linhas diferentes linhas, caso o trabalhador possua mais de um CBO cadastrado.

In [46]:
trab_com_cursos.PRETENSOES = trab_com_cursos.PRETENSOES.str.split('|')
trab_com_cursos = trab_com_cursos.explode('PRETENSOES')

# Removendo os registros com o campo PRETENSAO nulo

In [63]:
trab_com_cursos = trab_com_cursos[trab_com_cursos.PRETENSOES.notnull()]

# Criando um campo com Código da Ocupação a partir do campo PRETENSOES

In [67]:
trab_com_cursos['cod_ocupacao'] = trab_com_cursos.PRETENSOES.str.split('-').apply(lambda x:x[0])

In [68]:
trab_com_cursos

,NACIONALIDADE,DEFICIENCIAS,BAIRRO,CEP,CODIGO_MUNICIPIO_IBGE,NOME_MUNICIPIO,UF,ESCOLARIDADE,ESTUDANTE,CURSOS_PROFISSIONALIZANTES,...,POS_GRADUACOES,IDIOMAS,HABILITACAO,VEICULOS,DISP_VIAJAR,DISP_DORMIR_EMP,DISP_AUSENTAR_DOMIC,PRETENSOES,MUNICIPIOS_INTERESSE,cod_ocupacao
14,BRASILEIRA,NaN,ISAURA PARENTE,6.99183e+07,120040,RIO BRANCO,AC,Superior Completo,N,aperfeicoamento em educacao infantil|curso bas...,...,NaN,NaN,AB,S,S,S,S,232115-PROFESSOR DE DISCIPLINAS PEDAGOGICAS NO...,NaN,232115
14,BRASILEIRA,NaN,ISAURA PARENTE,6.99183e+07,120040,RIO BRANCO,AC,Superior Completo,N,aperfeicoamento em educacao infantil|curso bas...,...,NaN,NaN,AB,S,S,S,S,"239415-Pedagogo(N,0,6,Comercial)",NaN,239415
14,BRASILEIRA,NaN,ISAURA PARENTE,6.99183e+07,120040,RIO BRANCO,AC,Superior Completo,N,aperfeicoamento em educacao infantil|curso bas...,...,NaN,NaN,AB,S,S,S,S,"354145-Vendedor pracista(N,19,0,Comercial)",NaN,354145
14,BRASILEIRA,NaN,ISAURA PARENTE,6.99183e+07,120040,RIO BRANCO,AC,Superior Completo,N,aperfeicoamento em educacao infantil|curso bas...,...,NaN,NaN,AB,S,S,S,S,"521110-Vendedor interno(N,0,0,Indiferente)",NaN,521110
14,BRASILEIRA,NaN,ISAURA PARENTE,6.99183e+07,120040,RIO BRANCO,AC,Superior Completo,N,aperfeicoamento em educacao infantil|curso bas...,...,NaN,NaN,AB,S,S,S,S,"524105-Vendedor de serviços(N,0,36,Comercial)",NaN,524105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13649686,BRASILEIRA,NaN,Setor Sul Taquaralto,7.70646e+07,172100,PALMAS,TO,Médio Completo,N,manutençao de computadores|operador de maquina...,...,NaN,NaN,Nenhum,N,N,N,N,"521125-Repositor de mercadorias(N,0,6,Indifere...",NaN,521125
13649686,BRASILEIRA,NaN,Setor Sul Taquaralto,7.70646e+07,172100,PALMAS,TO,Médio Completo,N,manutençao de computadores|operador de maquina...,...,NaN,NaN,Nenhum,N,N,N,N,"782220-Operador de empilhadeira(N,0,6,Indifere...",NaN,782220
13649686,BRASILEIRA,NaN,Setor Sul Taquaralto,7.70646e+07,172100,PALMAS,TO,Médio Completo,N,manutençao de computadores|operador de maquina...,...,NaN,NaN,Nenhum,N,N,N,N,"784205-Auxiliar de linha de produção(N,0,6,Ind...",NaN,784205
13649686,BRASILEIRA,NaN,Setor Sul Taquaralto,7.70646e+07,172100,PALMAS,TO,Médio Completo,N,manutençao de computadores|operador de maquina...,...,NaN,NaN,Nenhum,N,N,N,N,"951105-Eletricista(N,0,6,Indiferente)",NaN,951105


In [69]:
trab_com_cursos.to_pickle("./trabalhadores.pkl")

In [ ]:
# trab_com_cursos = pd.read_pickle("./trabalhadores.pkl")

# Extraindo as 5 principais ocupações

In [76]:
top_5_ocup = list(trab_com_cursos.cod_ocupacao.value_counts().index[:5])

In [104]:
print(f'As ocupações mais procuradas pelos trabalhadores são: {top_5_ocup}')

As ocupações mais procuradas pelos trabalhadores são: ['784205', '521110', '411005', '422105', '421125']


# Criando o corpus apenas para a principal ocupação

In [80]:
corpus = trab_com_cursos.loc[trab_com_cursos.cod_ocupacao == top_5_ocup[0],'CURSOS_PROFISSIONALIZANTES'].copy()

In [82]:
corpus = preprocess_corpus(corpus)

100%|██████████| 705010/705010 [05:09<00:00, 2279.95it/s]


In [84]:
def get_top_n_words(corpus, n=5, n_gram=2):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer(ngram_range=(n_gram,n_gram)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [85]:
get_top_n_words(corpus)

[('informatica basica', 200414),
 ('operador empilhadeira', 38199),
 ('auxiliar administrativo', 21199),
 ('informatica basico', 19923),
 ('seguranca trabalho', 15808)]

# Lendo e salvando todos os arquivos de vagas em um único dataframe

In [87]:
path = r'SINE/Vagas' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, sep=";", encoding='iso-8859-1', usecols = list(range(12)))
    li.append(df)

vagas = pd.concat(li, axis=0, ignore_index=True)

100%|██████████| 27/27 [00:00<00:00, 104.78it/s]


# Eliminando todos os registros que possuem o campo OBSERVACOES_OCUPACAO nulo.

In [91]:
vagas = vagas[vagas.OBSERVACOES_OCUPACAO.notnull()]

In [92]:
vagas

,CODIGO_OCUPACAO,TITULO_OCUPACAO,OBSERVACOES_OCUPACAO,CODIGO_MUNICIPIO_IBGE,CEP,VALOR_SALARIO,QTD_VAGAS,TIPO_CONTRATACAO,QTD_EXPERIENCIA,ESCOLARIDADE,REQUERIDO_DEFICIENCIA,DEFICIENCIAS
1,514325,Oficial de serviços diversos na manutenção de ...,VAGA EXCLUSIVA PARA PESSOA COM DEFICIÊNCIA,270030,57300970,0,1.0,3 - Permanente,6.0,6 - Médio Incompleto,0.0,0 - Surdez bilateral parcial - Auditiva|1 - Su...
4,411005,Auxiliar de escritório,Trabalho a ser executado sera o de preparacao ...,291955,47850000,0,80.0,3 - Permanente,0.0,7 - Médio Completo,0.0,0 - Surdez bilateral parcial - Auditiva|1 - Su...
8,411005,Auxiliar administrativo,VAGA EXCLUSIVA SOBRADINHO BA,293077,48925000,0,10.0,2 - Temporária,6.0,7 - Médio Completo,0.0,11 - Parcial - Física
10,782320,Condutor de ambulância,"VAGA EXCLUSIVA PARA SOBRADINHO-BA, TRABALHADOR...",293077,48925000,0,1.0,2 - Temporária,6.0,7 - Médio Completo,0.0,11 - Parcial - Física
11,311505,Técnico de gestão do meio ambiente,"VAGA EXCLUSIVA PARA SOBRADINHO-BA, TRABALHADOR...",293077,48925000,0,1.0,2 - Temporária,6.0,7 - Médio Completo,0.0,11 - Parcial - Física
...,...,...,...,...,...,...,...,...,...,...,...,...
2196,318805,Projetista de móveis,vaga para projetista de moveis com experiência...,170210,77803060,0,1.0,3 - Permanente,6.0,7 - Médio Completo,0.0,11 - Parcial - Física
2197,623110,Vaqueiro,vai cuidar de tudo na fazenda. fazenda próximo...,172100,77020016,1045,1.0,3 - Permanente,6.0,10 - Fundamental Incompleto,0.0,0 - Surdez bilateral parcial - Auditiva
2198,514325,Oficial de serviços gerais na manutenção de ed...,irá executar serviços de limpeza em geral. ter...,172100,77061900,1045,1.0,3 - Permanente,6.0,0 - Nenhum,0.0,0 - Surdez bilateral parcial - Auditiva
2199,514325,Oficial de serviços gerais na manutenção de ed...,é NECESSÁRIO que o candidato tenha EXPERIENC...,172100,77061900,1045,1.0,3 - Permanente,0.0,10 - Fundamental Incompleto,0.0,0 - Surdez bilateral parcial - Auditiva| 13 - ...


# Criando o corpus apenas para vagas da principal ocupação

In [94]:
corpus_vagas = vagas.loc[vagas.CODIGO_OCUPACAO == top_5_ocup[0],'OBSERVACOES_OCUPACAO'].copy()

In [95]:
corpus_vagas = preprocess_corpus(corpus_vagas)

100%|██████████| 2/2 [00:00<00:00, 1001.74it/s]


In [96]:
get_top_n_words(corpus_vagas)

[('pcd organizarempilharmolhardobrar', 1),
 ('organizarempilharmolhardobrar ajustar', 1),
 ('ajustar couros', 1),
 ('couros pallet', 1),
 ('pallet mesaconferir', 1)]